In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
import time
import os

In [7]:
# Generate new data, if needed

X_baseline = [] # baseline time point
for filename in sorted(os.listdir("ADNIMERGE/baseline/")):
    ct = np.loadtxt("ADNIMERGE/baseline/"+filename, delimiter='\n')
    X_baseline.append(ct)
    
X_followup = [] # other longitudinal time point
for filename in sorted(os.listdir("ADNIMERGE/followup/")):
    ct = np.loadtxt("ADNIMERGE/followup/"+filename, delimiter='\n')
    X_followup.append(ct)

print(len(X_baseline), len(X_followup))
np.save("X_baseline.npy", np.array(X_baseline))
np.save("X_followup.npy", np.array(X_followup))

1060 1060


In [3]:
# Generate the AIBL replication data

aibl_baseline = []
for filename in sorted(os.listdir("AIBL/baseline/")):
    ct = np.loadtxt("AIBL/baseline/"+filename, delimiter='\n')
    aibl_baseline.append(ct)
    
aibl_followup = [] # other longitudinal time point
for filename in sorted(os.listdir("AIBL/followup/")):
    ct = np.loadtxt("AIBL/followup/"+filename, delimiter='\n')
    aibl_followup.append(ct)

print(len(aibl_baseline), len(aibl_followup))
np.save("aibl_baseline.npy", np.array(aibl_baseline))
np.save("aibl_followup.npy", np.array(aibl_followup))

113 113


In [4]:
# Load the data

df = pd.read_csv("Exp_502_602_combined.csv")
sub_list = df[df["STATUS"].isin(["OK", "PrevTP", "New"])]
X_baseline = np.load("X_baseline.npy")
X_followup = np.load("X_followup.npy")
aibl_baseline = np.load("aibl_baseline.npy")
aibl_followup = np.load("aibl_followup.npy")
X_diff = X_followup - X_baseline # vertex-wise CT change
n_subjects = sub_list.shape[0]
print(n_subjects, X_baseline.shape, X_followup.shape, aibl_baseline.shape, aibl_followup.shape)

1060 (1060, 81924) (1060, 81924) (113, 81924) (113, 81924)


In [14]:
# Generate the train-test splits (old script)

frac_train = 0.8
ncv = 10
splits = {"train": [], "test": []}

for i in range(ncv):
    indices = list(range(n_subjects))
    random.shuffle(indices)
    train_split = indices[:int(0.8*len(indices))]
    valid_split = indices[int(0.8*len(indices)):]
    splits["train"].append(train_split)
    splits["test"].append(valid_split)
    
splits = pd.DataFrame.from_dict(splits)
splits.to_pickle("train_test_splits.pkl")

In [10]:
# Generate the train-test splits (modified)
# Splits balanced with respect to trajectory classes and subgroups

ncv = 10
frac_test = 1/ncv
splits = {'MMSE': {'train': [], 'test': []},
          'ADAS13': {'train': [], 'test': []}}
indices = {'MMSE': {'T1': {'BE': [], 'FE': [], 'CC': []},
                    'T2': {'BE': [], 'FE': [], 'CC': []}},
           'ADAS13': {'T1': {'BE': [], 'FE': [], 'CC': []},
                      'T2': {'BE': [], 'FE': [], 'CC': []},
                      'T3': {'BE': [], 'FE': [], 'CC': []}}}

sub_list = sub_list.reset_index(drop=True)

# find the trajectory and subgroup of each index
for index, row in sub_list.iterrows():
    mmse_traj = row['MMSE_2c_traj']
    mmse_group = row['MMSE_gr']
    indices['MMSE']['T{}'.format(mmse_traj+1)][mmse_group].append(index)
    
    adas_traj = row['ADAS_3c_traj']
    adas_group = row['ADAS13_gr']
    indices['ADAS13']['T{}'.format(adas_traj+1)][adas_group].append(index)

splits_mmse = {'train': [], 'test': []}
splits_adas = {'train': [], 'test': []}
    
# partition into balanced train and test splits
for i in indices.keys():
    train_split = [[] for cv_iter in range(ncv)]
    test_split = [[] for cv_iter in range(ncv)]
    
    for j in indices[i].keys():
        for k in indices[i][j].keys():
            random.shuffle(indices[i][j][k])
            length = len(indices[i][j][k])
            
            for cv_iter in range(ncv):
                start = int(length*cv_iter*frac_test)
                end = int(length*(cv_iter+1)*frac_test)
                test_split[cv_iter].extend(indices[i][j][k][start:end])
                before = indices[i][j][k][:start]
                after = indices[i][j][k][end:]
                train_split[cv_iter].extend(before + after)
    
    for cv_iter in range(ncv):
        random.shuffle(train_split[cv_iter])
        random.shuffle(test_split[cv_iter])
    
    splits[i]['train'] = train_split
    splits[i]['test'] = test_split
        
splits = pd.DataFrame.from_dict(splits)
splits.to_pickle("train_test_splits.pkl")

In [11]:
# Reduce the data with PCA
from sklearn.decomposition import PCA

ncv = 10
n_components = 78 # same number of components as AAL
splits = pd.read_pickle("train_test_splits.pkl")

for t in ['MMSE', 'ADAS13']:
    for i in range(ncv):
        print("Reducing data, {} trajectory, fold {}".format(t, i))
        train_split = splits[t]['train'][i]
        test_split = splits[t]['test'][i]
        X_train_baseline = X_baseline[train_split]
        X_test_baseline = X_baseline[test_split]
        X_train_followup = X_followup[train_split]
        X_test_followup = X_followup[test_split]
        X_train_diff = X_diff[train_split]

        tstart = time.time()
        pca = PCA(n_components=n_components)
        pca.fit(X_train_diff)
        X_bl_train_reduced = pca.transform(X_train_baseline)
        X_bl_test_reduced = pca.transform(X_test_baseline)
        X_vartp_train_reduced = pca.transform(X_train_followup)
        X_vartp_test_reduced = pca.transform(X_test_followup)
        print("Time required : {}".format(time.time() - tstart))
        np.save("data/PCA_bl_train_{}_cv{}.npy".format(t,i), X_bl_train_reduced)
        np.save("data/PCA_bl_test_{}_cv{}.npy".format(t,i), X_bl_test_reduced)
        np.save("data/PCA_vartp_train_{}_cv{}.npy".format(t,i), X_vartp_train_reduced)
        np.save("data/PCA_vartp_test_{}_cv{}.npy".format(t,i), X_vartp_test_reduced)
        
        if t == 'MMSE': # Reduce the AIBL replication data
            aibl_bl_reduced = pca.transform(aibl_baseline)
            aibl_vartp_reduced = pca.transform(aibl_followup)
            np.save("data_replication/PCA_bl_cv{}.npy".format(i), aibl_bl_reduced)
            np.save("data_replication/PCA_vartp_cv{}.npy".format(i), aibl_vartp_reduced)

Reducing data, MMSE trajectory, fold 0
Time required : 24.295142650604248
Reducing data, MMSE trajectory, fold 1
Time required : 15.013468980789185
Reducing data, MMSE trajectory, fold 2
Time required : 15.413044214248657
Reducing data, MMSE trajectory, fold 3
Time required : 15.253373861312866
Reducing data, MMSE trajectory, fold 4
Time required : 15.392229557037354
Reducing data, MMSE trajectory, fold 5
Time required : 14.515857934951782
Reducing data, MMSE trajectory, fold 6
Time required : 13.738002061843872
Reducing data, MMSE trajectory, fold 7
Time required : 14.657521486282349
Reducing data, MMSE trajectory, fold 8
Time required : 14.870039939880371
Reducing data, MMSE trajectory, fold 9
Time required : 14.800063610076904
Reducing data, ADAS13 trajectory, fold 0
Time required : 14.366201877593994
Reducing data, ADAS13 trajectory, fold 1
Time required : 14.074681997299194
Reducing data, ADAS13 trajectory, fold 2
Time required : 14.113887548446655
Reducing data, ADAS13 trajectory

In [8]:
# Reduce the data with RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

n_features = 78 # same number of components as AAL
splits = pd.read_pickle("train_test_splits.pkl")

for t in ['MMSE', 'ADAS13']:
    for i in range(ncv):
        print("Reducing data, {} trajectory, fold {}".format(t, i))
        train_split = splits[t]['train'][i]
        test_split = splits[t]['test'][i]
        X_train_baseline = X_baseline[train_split]
        X_test_baseline = X_baseline[test_split]
        X_train_followup = X_followup[train_split]
        X_test_followup = X_followup[test_split]
        X_train_diff = X_diff[train_split]

        df_train = sub_list.iloc[train_split]
        if t == 'MMSE':
            y_train = df_train["MMSE_2c_traj"].values
        elif t == 'ADAS13':
            y_train = df_train["ADAS_3c_traj"].values

        tstart = time.time()
        estimator = SVC(kernel='linear')
        rfe = RFE(estimator, n_features, step=0.5)
        rfe.fit(X_train_diff, y_train)
        X_bl_train = rfe.transform(X_train_baseline)
        X_bl_test = rfe.transform(X_test_baseline)
        X_vartp_train = rfe.transform(X_train_followup)
        X_vartp_test = rfe.transform(X_test_followup)
        print("Time required : {}".format(time.time() - tstart))
        np.save("data/RFE_bl_train_{}_cv{}.npy".format(t,i), X_bl_train)
        np.save("data/RFE_bl_test_{}_cv{}.npy".format(t,i), X_bl_test)
        np.save("data/RFE_vartp_train_{}_cv{}.npy".format(t,i), X_vartp_train)
        np.save("data/RFE_vartp_test_{}_cv{}.npy".format(t,i), X_vartp_test)
        
        if t == 'MMSE': # Reduce the AIBL replication data
            aibl_bl_reduced = rfe.transform(aibl_baseline)
            aibl_vartp_reduced = rfe.transform(aibl_followup)
            np.save("data_replication/RFE_bl_cv{}.npy".format(i), aibl_bl_reduced)
            np.save("data_replication/RFE_vartp_cv{}.npy".format(i), aibl_vartp_reduced)

Reducing data, MMSE trajectory, fold 0
Time required : 152.86832427978516
Reducing data, MMSE trajectory, fold 1
Time required : 160.58308005332947
Reducing data, MMSE trajectory, fold 2
Time required : 150.17101907730103
Reducing data, MMSE trajectory, fold 3
Time required : 149.13540387153625
Reducing data, MMSE trajectory, fold 4
Time required : 163.4870743751526
Reducing data, MMSE trajectory, fold 5
Time required : 149.81610465049744
Reducing data, MMSE trajectory, fold 6
Time required : 149.00684571266174
Reducing data, MMSE trajectory, fold 7
Time required : 160.2220332622528
Reducing data, MMSE trajectory, fold 8
Time required : 171.1578848361969
Reducing data, MMSE trajectory, fold 9
Time required : 147.49310183525085
Reducing data, ADAS13 trajectory, fold 0
Time required : 209.52611994743347
Reducing data, ADAS13 trajectory, fold 1
Time required : 206.6359670162201
Reducing data, ADAS13 trajectory, fold 2
Time required : 221.22635412216187
Reducing data, ADAS13 trajectory, fo

In [ ]:
# Reduce the data with RLR
# Idea by Moradi et al., 2015
from sklearn.linear_model import SGDClassifier

n_features = 78 # same number of components as AAL
splits = pd.read_pickle("train_test_splits.pkl")

n_repeats = 5 # 10 by Moradi et al.
range_size = 10 # 100 by Moradi et al.
alpha_range = np.power(10, np.linspace(-5, -2, range_size))
print("Range of alphas: {}".format(alpha_range))

for t in ['MMSE', 'ADAS13']:
    for i in range(ncv):
        train_split = splits[t]["train"][i]
        test_split = splits[t]["test"][i]
        X_train_baseline = X_baseline[train_split]
        X_test_baseline = X_baseline[test_split]
        X_train_followup = X_followup[train_split]
        X_test_followup = X_followup[test_split]
        X_train_diff = X_diff[train_split]

        df_train = sub_list.iloc[train_split]
        if t == 'MMSE':
            y_train = df_train["MMSE_2c_traj"]
        elif t == 'ADAS13':
            y_train = df_train["ADAS_3c_traj"]

        print("Begin {}, CV fold number {}".format(t,i))
        print("Step one: choose the optimal alpha")
        tstart = time.time()
        alpha_star = []

        for j in range(n_repeats):
            print("Beginning repeat {}...".format(j))
            scores = np.zeros(range_size)

            indices = list(range(len(train_split)))
            random.shuffle(indices)
            inner_train = indices[:int(0.9*len(indices))]
            inner_valid = indices[int(0.9*len(indices)):]
            X_diff_inner_train = X_diff[inner_train]
            X_diff_inner_valid = X_diff[inner_valid]
            df_inner_train = df_train.iloc[inner_train]
            df_inner_valid = df_train.iloc[inner_valid]
            if t == 'MMSE':
                y_inner_train = df_inner_train["MMSE_2c_traj"]
                y_inner_valid = df_inner_valid["MMSE_2c_traj"]
            elif t == 'ADAS13':
                y_inner_train = df_inner_train["ADAS_3c_traj"]
                y_inner_valid = df_inner_valid["ADAS_3c_traj"]

            for k in range(range_size):
                clf = SGDClassifier(loss="log", penalty="elasticnet", l1_ratio=0.5, alpha=alpha_range[k])
                clf.fit(X_diff_inner_train, y_inner_train)
                scores[k] = clf.score(X_diff_inner_valid, y_inner_valid)

            alpha_star.append(alpha_range[np.argmax(scores)])

        print("Optimal alpha: {}".format(np.median(alpha_star)))
        print("Step two: find the most significant features")
        k = np.where(alpha_range <= np.median(alpha_star))[0][-1]
        nonzero_features = np.ones(X_baseline.shape[1])

        for alpha in alpha_range[k:]:
            clf = SGDClassifier(loss="log", penalty="elasticnet",
                                l1_ratio=0.5, alpha=alpha)
            clf.fit(X_train_diff, y_train)
            nonzero_features = np.logical_and(nonzero_features, (clf.coef_ > 0))

        nonzero_indices = np.where(nonzero_features)[1]
        X_train_diff_remain = X_train_diff[:, nonzero_indices]
        print("Number of nonzero features: {}".format(nonzero_indices.shape[0]))
        coef_sums = np.zeros(X_train_diff_remain.shape[1])

        for j in range(n_repeats):
            clf = SGDClassifier(loss="log", penalty="l2", alpha=alpha_range[k])
            clf.fit(X_train_diff_remain, y_train)
            coef_sums = coef_sums + clf.coef_

        most_significant = np.argsort(coef_sums)[0, :78]
        features_selected = nonzero_indices[most_significant]

        X_bl_train = X_train_baseline[:, features_selected]
        X_bl_test = X_test_baseline[:, features_selected]
        X_vartp_train = X_train_followup[:, features_selected]
        X_vartp_test = X_test_followup[:, features_selected]
        print("Time required : {}".format(time.time() - tstart))
        np.save("data/RLR_bl_train_{}_cv{}.npy".format(t,i), X_bl_train)
        np.save("data/RLR_bl_test_{}_cv{}.npy".format(t,i), X_bl_test)
        np.save("data/RLR_vartp_train_{}_cv{}.npy".format(t,i), X_vartp_train)
        np.save("data/RLR_vartp_test_{}_cv{}.npy".format(t,i), X_vartp_test)
        
        if t == 'MMSE': # Reduce the AIBL replication data
            aibl_bl_reduced = aibl_baseline[:, features_selected]
            aibl_vartp_reduced = aibl_followup[:, features_selected]
            np.save("data_replication/RLR_bl_cv{}.npy".format(i), aibl_bl_reduced)
            np.save("data_replication/RLR_vartp_cv{}.npy".format(i), aibl_vartp_reduced)

Range of alphas: [1.00000000e-05 2.15443469e-05 4.64158883e-05 1.00000000e-04
 2.15443469e-04 4.64158883e-04 1.00000000e-03 2.15443469e-03
 4.64158883e-03 1.00000000e-02]
Begin MMSE, CV fold number 0
Step one: choose the optimal alpha
Beginning repeat 0...


/opt/quarantine/anaconda/5.1.0-python3/install/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3...
Beginning repeat 4...
Optimal alpha: 0.0001
Step two: find the most significant features
Number of nonzero features: 98
Time required : 278.84364342689514
Begin MMSE, CV fold number 1
Step one: choose the optimal alpha
Beginning repeat 0...
Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3...
Beginning repeat 4...
Optimal alpha: 0.001
Step two: find the most significant features
Number of nonzero features: 72
Time required : 261.72918248176575
Begin MMSE, CV fold number 2
Step one: choose the optimal alpha
Beginning repeat 0...
Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3...
Beginning repeat 4...
Optimal alpha: 0.00046415888336127773
Step two: find the most significant features
Number of nonzero features: 101
Time required : 266.99633741378784
Begin MMSE, CV fold number 3
Step one: choose the optimal alpha
Beginning repeat 0...
Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3.

In [9]:
# Reduce the data with HCA
from sklearn.cluster import AgglomerativeClustering

n_features = 78 # same number of components as AAL
n_partitions = 20 # 2-step HCA process
partition_size = 40962/n_partitions
n_clusters = int(partition_size/n_partitions)
splits = pd.read_pickle("train_test_splits.pkl")

for t in ['MMSE', 'ADAS13']:
    for i in range(ncv):
        train_split = splits[t]["train"][i]
        test_split = splits[t]["test"][i]
        X_train_baseline = X_baseline[train_split]
        X_test_baseline = X_baseline[test_split]
        X_train_followup = X_followup[train_split]
        X_test_followup = X_followup[test_split]
        X_train_diff = X_diff[train_split]

        tstart = time.time()
        print("Beginning HCA, {} trajectory, CV fold number {}".format(t,i))
        print("Clustering left vertices...")

        left_vertices = X_train_diff.T[:40962, :]
        left_vertices_merged = []
        left_cluster_list = []

        for p in range(n_partitions):
            vp = left_vertices[int(p*partition_size):int((p+1)*partition_size)]
            hca = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
            clustering = hca.fit(vp)

            for cluster in np.unique(clustering.labels_):
                ind = np.where(clustering.labels_ == cluster)[0]
                left_cluster_list.append(ind + int(p*partition_size))
                left_vertices_merged.append(np.mean(vp[ind, :], axis=0))

        hca = AgglomerativeClustering(n_clusters=int(np.floor(n_features/2)), linkage="ward")
        left_clustering = hca.fit(np.array(left_vertices_merged))

        print("Clustering right vertices...")

        right_vertices = X_train_diff.T[-40962:, :]
        right_vertices_merged = []
        right_cluster_list = []

        for p in range(n_partitions):
            vp = right_vertices[int(p*partition_size):int((p+1)*partition_size)]
            hca = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
            clustering = hca.fit(vp)

            for cluster in np.unique(clustering.labels_):
                ind = np.where(clustering.labels_ == cluster)[0]
                right_cluster_list.append(ind + int(p*partition_size))
                right_vertices_merged.append(np.mean(vp[ind, :], axis=0))

        hca = AgglomerativeClustering(n_clusters=int(np.ceil(n_features/2)), linkage="ward")
        right_clustering = hca.fit(np.array(right_vertices_merged))

        print("Reducing the data...")

        X_bl_train_reduced = np.zeros((X_train_baseline.shape[0], n_features))
        X_bl_test_reduced = np.zeros((X_test_baseline.shape[0], n_features))
        X_vartp_train_reduced = np.zeros((X_train_followup.shape[0], n_features))
        X_vartp_test_reduced = np.zeros((X_test_followup.shape[0], n_features))
        
        if t == 'MMSE': # Reduce the AIBL replication data
            aibl_bl_reduced = np.zeros((aibl_baseline.shape[0], n_features))
            aibl_vartp_reduced = np.zeros((aibl_followup.shape[0], n_features))

        for cluster in np.unique(left_clustering.labels_):
            subclusters = np.where(left_clustering.labels_ == cluster)[0]
            ind = np.concatenate([left_cluster_list[sc] for sc in subclusters])
            X_bl_train_reduced[:, cluster] = np.mean(X_train_baseline[:, ind], axis=1)
            X_bl_test_reduced[:, cluster] = np.mean(X_test_baseline[:, ind], axis=1)
            X_vartp_train_reduced[:, cluster] = np.mean(X_train_followup[:, ind], axis=1)
            X_vartp_test_reduced[:, cluster] = np.mean(X_test_followup[:, ind], axis=1)
            
            if t == 'MMSE': 
                aibl_bl_reduced[:, cluster] = np.mean(aibl_baseline[:, ind], axis=1)
                aibl_vartp_reduced[:, cluster] = np.mean(aibl_followup[:, ind], axis=1)

        shift = int(np.floor(n_features/2))
        for cluster in np.unique(right_clustering.labels_):
            subclusters = np.where(right_clustering.labels_ == cluster)[0]
            ind = np.concatenate([right_cluster_list[sc] for sc in subclusters])
            X_bl_train_reduced[:, cluster + shift] = np.mean(X_train_baseline[:, ind], axis=1)
            X_bl_test_reduced[:, cluster + shift] = np.mean(X_test_baseline[:, ind], axis=1)
            X_vartp_train_reduced[:, cluster + shift] = np.mean(X_train_followup[:, ind], axis=1)
            X_vartp_test_reduced[:, cluster + shift] = np.mean(X_test_followup[:, ind], axis=1)
            
            if t == 'MMSE': 
                aibl_bl_reduced[:, cluster + shift] = np.mean(aibl_baseline[:, ind], axis=1)
                aibl_vartp_reduced[:, cluster + shift] = np.mean(aibl_followup[:, ind], axis=1)

        print("Time required : {}".format(time.time() - tstart))
        np.save("data/HCA_bl_train_{}_cv{}.npy".format(t,i), X_bl_train_reduced)
        np.save("data/HCA_bl_test_{}_cv{}.npy".format(t,i), X_bl_test_reduced)
        np.save("data/HCA_vartp_train_{}_cv{}.npy".format(t,i), X_vartp_train_reduced)
        np.save("data/HCA_vartp_test_{}_cv{}.npy".format(t,i), X_vartp_test_reduced)
        
        if t == 'MMSE': # Reduce the AIBL replication data
            np.save("data_replication/HCA_bl_cv{}.npy".format(i), aibl_bl_reduced)
            np.save("data_replication/HCA_vartp_cv{}.npy".format(i), aibl_vartp_reduced)

Beginning HCA, MMSE trajectory, CV fold number 0
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 80.25991988182068
Beginning HCA, MMSE trajectory, CV fold number 1
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 75.86690258979797
Beginning HCA, MMSE trajectory, CV fold number 2
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 78.47535967826843
Beginning HCA, MMSE trajectory, CV fold number 3
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 77.66696286201477
Beginning HCA, MMSE trajectory, CV fold number 4
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 74.2001678943634
Beginning HCA, MMSE trajectory, CV fold number 5
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 76.31808352470398
Beginning HCA, MMSE trajectory, CV 